# Pandas Grouper

`Grouper`permite crear `groupby`con instrucciones mas precisas. Tiene una utilidad importante en el manejo de fechas, por ejemplo, en la agrupación de datos generando intervalos de fechas con la posibilidad de indicar el inicio, el final y el paso en diferentes unidades: minutos, horas, días, etc.

<b> Creado: Enero 18 de 2021 <br>
Modificado: Junio 4 de 2021 <br>
Testeado en:

    - Python 3.8.6
    - pandas 1.1.3

In [1]:
import pandas as pd
import numpy as np

from IPython.display import display_html


def display_inline(*dataframes):
    df_html = [df.to_html() for df in dataframes]
    html = ''.join(df_html)
    html = html.replace('table','table style="display:inline; padding:10px"')
    display_html(html, raw=True)

## Grouper sobre datos categóricos

In [2]:
df = pd.DataFrame({
    'Nombre': ['Paul', 'Alicia', 'Murakami', 'Paul', 'Alicia'],
    'Cantidad': [23, 2, 12, 34, 6]
})

df

,Nombre,Cantidad
0,Paul,23
1,Alicia,2
2,Murakami,12
3,Paul,34
4,Alicia,6


In [3]:
group = pd.Grouper(key='Nombre')
df.groupby(group).sum()

,Cantidad
Nombre,
Paul,57
Alicia,8
Murakami,12


La variable `group` es entonces un objeto de tipo `Grouper` sobre la columna (axis=0) Nombre (key='Nombre') y sin ordenamiento (Sort=False).

In [4]:
group

Grouper(key='Nombre', axis=0, sort=False)

Estos parámetros puedes ser extraidos de la variable `group` como atributos del objeto `Grouper`.

In [5]:
group.key

'Nombre'

In [6]:
group.axis

0

In [7]:
group.sort

False

## Intervalos con fechas

Una de las funciones utilizadas en pandas para crear intervalos de fechas es `date_range`. Se crea un intervalo dada la fecha inicial y la fecha final con una frecuencia de 20 min:

In [8]:
inicio = '2020-05-02 4:24:00'
fin = '2020-05-02 8:17:00'

rango = pd.date_range(inicio, fin, freq='20min')
df = pd.DataFrame(rango, columns=['Fecha'])

df

,Fecha
0,2020-05-02 04:24:00
1,2020-05-02 04:44:00
2,2020-05-02 05:04:00
3,2020-05-02 05:24:00
4,2020-05-02 05:44:00
5,2020-05-02 06:04:00
6,2020-05-02 06:24:00
7,2020-05-02 06:44:00
8,2020-05-02 07:04:00
9,2020-05-02 07:24:00


Ahora se pretende realizar un conteo, por hora, de los registros del intervalo creado anteriormente. Para esto se crea un `Grouper` indicando el nombre de la columna *Fecha* y se especifica la frecuencia en horas usando `freq='H'`.

In [9]:
df.groupby(pd.Grouper(key='Fecha', freq='H')).size()

Fecha
2020-05-02 04:00:00    2
2020-05-02 05:00:00    3
2020-05-02 06:00:00    3
2020-05-02 07:00:00    3
2020-05-02 08:00:00    1
Freq: H, dtype: int64

Se realiza un ejercicio análogo con una frecuencia de 30 minutos.

In [10]:
df.groupby(pd.Grouper(key='Fecha', freq='30min')).size()

Fecha
2020-05-02 04:00:00    1
2020-05-02 04:30:00    1
2020-05-02 05:00:00    2
2020-05-02 05:30:00    1
2020-05-02 06:00:00    2
2020-05-02 06:30:00    1
2020-05-02 07:00:00    2
2020-05-02 07:30:00    1
2020-05-02 08:00:00    1
Freq: 30T, dtype: int64

El argumento `offset` permite agregarle un tiempo al origen de la serie y el origen viene definido en el argumento `origin` que por defecto es `start_day` en el cual se considera el primer día a media noche de la serie que se está agrupando. Consideremos el siguiente ejemplo sobre el dataframe `df`: 

- `origin='start'` indica que se tendrá como referencia el primer valor de la serie de tiempo, en este caso corresponde a *2020-05-02 4:24:00*,
- `offset='2min'` con lo que se le agregan estos dos minutos al primer valor de la serie de tiempo lo que da como resultado *2020-05-02 4:26:00*,
- `freq='30min'` indica la frecuencia de la serie.

Noten que al aumentar 2 minutos sobre el valor inicial de la serie esl primer valor (*2020-05-02 4:24:00*) es menor que el dado por el offset *2020-05-02 4:26:00* por lo que debe toamrse un valor previo utilizando la frecuencia. En resumen:

- Primer valor de la serie original: 2020-05-02 4:24:00
- Valor mas el offset: 2020-05-02 4:24:00 + 2min = 2020-05-02 4:26:00
- Valor inicial en la serie agrupada: 2020-05-02 4:26:00 - 30min = 2020-05-02 03:56:00

In [11]:
group = pd.Grouper(key='Fecha', freq='30min', offset='2min', origin='start')
df.groupby(group).size()

Fecha
2020-05-02 03:56:00    1
2020-05-02 04:26:00    1
2020-05-02 04:56:00    2
2020-05-02 05:26:00    1
2020-05-02 05:56:00    2
2020-05-02 06:26:00    1
2020-05-02 06:56:00    2
2020-05-02 07:26:00    1
2020-05-02 07:56:00    1
Freq: 30T, dtype: int64

Hay una diferencia al crear un `Grouper` sobre datos categóricos  y sobre fechas. Para este último se crea un objeto de tipo `TimeGrouper` que contiene aún mas atributos que pueden ser accedidos de la misma forma que sobre el objeto `Grouper`

In [12]:
group

TimeGrouper(key='Fecha', freq=<30 * Minutes>, axis=0, sort=True, closed='left', label='left', how='mean', convention='e', origin='start', offset=Timedelta('0 days 00:02:00'))

In [13]:
group.freq

<30 * Minutes>

In [14]:
group.offset

Timedelta('0 days 00:02:00')

## Diferencias con el groupby

In [15]:
df = pd.DataFrame({
    'Nombre': ['Paul', 'Alicia', 'Murakami', 'Paul', 'Alicia'],
    'Cantidad': [23, 2, 12, 34, 6]
})

df

,Nombre,Cantidad
0,Paul,23
1,Alicia,2
2,Murakami,12
3,Paul,34
4,Alicia,6


La diferencia de tiempo aún con tan pocos datos es considerable al realizar la misma operación y esto era de esperarse puesto que se opera sobre el objeto Gropuer, por lo que se recomineda utilizarlo unicamente cuando las característias del grupo a trabajar sean complejas y realmente se requiera especificar muchas cracterísticas de este como el inicio, fin, frecuenci, offset, etc.

In [16]:
group = pd.Grouper(key='Nombre')
%time df_grouper = df.groupby(group).sum()
%time df_groupby = df.groupby('Nombre').sum()

Wall time: 1.99 ms
Wall time: 1.99 ms


A diferencia del Grouper el groupby ordena por la columna de agrupamiento por defecto, pero esto puede desactivarse indicando como argumento `sort=False`

In [17]:
print('Grouper   \t              GroupBy\n')
display_inline(df_grouper, df_groupby)

Grouper   	              GroupBy



,Cantidad
Nombre,
Paul,57
Alicia,8
Murakami,12
,Cantidad
Nombre,
Alicia,8
Murakami,12
Paul,57
